In [1]:
!pip install sparqlwrapper
!pip install pandas

In [2]:
import os
try:
    os.mkdir('images')
except:
    pass

In [5]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil
import os
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT ?image WHERE {
  ?item wdt:P31 wd:Q8502 . # Check if item is a mountain
  ?item wdt:P18 ?image .
}
LIMIT 150
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def download_image(url):

    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        nom = os.path.basename(url)[-35:]
        chemin = "images/"+nom
        with open(chemin, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["image"]["value"],
        )
    )
    
dataframe = pd.DataFrame(array, columns=["image"])
dataframe = dataframe.astype(
    dtype={"image": "<U200"}
)
dataframe.image.apply(download_image)

dataframe

,image
0,http://commons.wikimedia.org/wiki/Special:File...
1,http://commons.wikimedia.org/wiki/Special:File...
2,http://commons.wikimedia.org/wiki/Special:File...
3,http://commons.wikimedia.org/wiki/Special:File...
4,http://commons.wikimedia.org/wiki/Special:File...
...,...
145,http://commons.wikimedia.org/wiki/Special:File...
146,http://commons.wikimedia.org/wiki/Special:File...
147,http://commons.wikimedia.org/wiki/Special:File...
148,http://commons.wikimedia.org/wiki/Special:File...


In [19]:
import os
import json
from PIL import Image, ExifTags, TiffImagePlugin

def extract_metadata(image_path):
  """
  Extracts metadata from a single image.

  Args:
      image_path: Path to the image file.

  Returns:
      A dictionary containing the extracted metadata.
  """
  try:
    image = Image.open(image_path)
    exif_data = image._getexif()
    metadata = dict()
    for key, value in exif_data.items():
      if isinstance(value, bytes):
        continue
      if (not isinstance(value, int)) and (not isinstance(value, float)):
        value = f"{value}"
      if key in ExifTags.TAGS:
        metadata[ExifTags.TAGS[key]] = value
    return metadata
  except:
    print(f"Error processing image: {image_path}")
    return {}


"""
Extracts metadata from all images in a folder and saves it to a JSON file.
"""
# Modify this path to your desired directory containing images
image_dir = "./images"

all_metadata = []
for filename in os.listdir(image_dir):
  if filename.lower().endswith((".jpg", ".jpeg", ".png")):
    image_path = os.path.join(image_dir, filename)
    metadata = extract_metadata(image_path)
    all_metadata.append(metadata)

# Modify this filename to your desired output file name
output_file = "image_metadata.json"

with open(output_file, "w") as f:
  json.dump(all_metadata, f, indent=4)

print(f"Image metadata saved to: {output_file}")

Error processing image: ./images\%20Crater%20Lake%20%28052005%29.jpg
Error processing image: ./images\%20geograph.org.uk%20-%20958044.jpg
Error processing image: ./images\0de%20la%20Barre%20des%20Ecrins.png
Error processing image: ./images\0do%20Monte%20Roraima%20%281%29.jpg
Error processing image: ./images\20Gabelhorn%20und%20Wellenkuppe.jpg
Error processing image: ./images\20geograph.org.uk%20-%201247806.jpg
Error processing image: ./images\20geograph.org.uk%20-%201456731.jpg
Error processing image: ./images\20geograph.org.uk%20-%201457518.jpg
Error processing image: ./images\20geograph.org.uk%20-%201657454.jpg
Error processing image: ./images\31MonteGeneroso.jpg
Error processing image: ./images\AiguilleduGeant0001.jpg
Error processing image: ./images\AiguillesPeuterey0002.jpg
Error processing image: ./images\Allalinhorn0001.jpg
Error processing image: ./images\Bos%20Fulen.jpg
Error processing image: ./images\Bur%20ni%20Telong.jpg
Error processing image: ./images\Drus-verte.jpg
Error